# Gap-Constrained Simulated Annealing

Implement SA optimization that maintains minimum gap between trees.
Focus on small N (1-20) first where search space is manageable.

In [1]:
import pandas as pd
import numpy as np
from shapely import affinity
from shapely.geometry import Polygon
from itertools import combinations
import json
import warnings
warnings.filterwarnings('ignore')

class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        
        initial_polygon = Polygon([
            (0.0, 0.8), (0.125, 0.5), (0.0625, 0.5),
            (0.2, 0.25), (0.1, 0.25), (0.35, 0.0),
            (0.075, 0.0), (0.075, -0.2), (-0.075, -0.2),
            (-0.075, 0.0), (-0.35, 0.0), (-0.1, 0.25),
            (-0.2, 0.25), (-0.0625, 0.5), (-0.125, 0.5),
        ])
        rotated = affinity.rotate(initial_polygon, self.angle, origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=self.center_x, yoff=self.center_y)

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

def get_min_distance(trees):
    if len(trees) <= 1:
        return float('inf')
    min_dist = float('inf')
    for i, j in combinations(range(len(trees)), 2):
        dist = trees[i].polygon.distance(trees[j].polygon)
        min_dist = min(min_dist, dist)
    return min_dist

def get_bounding_box_side(trees):
    all_points = []
    for tree in trees:
        coords = np.array(tree.polygon.exterior.coords)
        all_points.append(coords)
    all_points = np.vstack(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def is_valid_configuration(trees, min_gap=1e-9):
    if len(trees) <= 1:
        return True
    for i, j in combinations(range(len(trees)), 2):
        dist = trees[i].polygon.distance(trees[j].polygon)
        if dist < min_gap:
            return False
    return True

print("Functions defined")

Functions defined


In [2]:
# Gap-constrained SA optimizer
MIN_GAP = 0.001  # Minimum distance between trees

def has_gap_violation(positions, min_gap=MIN_GAP):
    """Check if any pair of trees violates minimum gap"""
    n = len(positions)
    if n <= 1:
        return False
    
    trees = [ChristmasTree(p[0], p[1], p[2]) for p in positions]
    for i in range(n):
        for j in range(i+1, n):
            if trees[i].polygon.distance(trees[j].polygon) < min_gap:
                return True
    return False

def gap_constrained_sa(n, initial_positions=None, iterations=5000, T0=1.0, Tm=0.0001):
    """
    Simulated annealing with gap constraints.
    Returns positions that maintain min_gap between all trees.
    """
    # Initialize positions
    if initial_positions is None:
        # Random initialization in a circle
        positions = []
        radius = 0.5 * np.sqrt(n)
        for i in range(n):
            angle = 2 * np.pi * i / n
            x = radius * np.cos(angle)
            y = radius * np.sin(angle)
            deg = np.random.uniform(0, 360)
            positions.append([x, y, deg])
    else:
        positions = [list(p) for p in initial_positions]
    
    # Ensure initial configuration is valid
    max_attempts = 100
    for attempt in range(max_attempts):
        if not has_gap_violation(positions):
            break
        # Spread out positions
        cx = sum(p[0] for p in positions) / n
        cy = sum(p[1] for p in positions) / n
        for p in positions:
            dx = p[0] - cx
            dy = p[1] - cy
            p[0] = cx + dx * 1.1
            p[1] = cy + dy * 1.1
    
    if has_gap_violation(positions):
        print(f"Warning: Could not create valid initial configuration for N={n}")
        return None
    
    # Calculate initial score
    def get_score(pos):
        trees = [ChristmasTree(p[0], p[1], p[2]) for p in pos]
        return get_bounding_box_side(trees)
    
    best_positions = [list(p) for p in positions]
    best_score = get_score(best_positions)
    current_positions = [list(p) for p in positions]
    current_score = best_score
    
    T = T0
    alpha = (Tm / T0) ** (1.0 / iterations)
    
    move_scale = 0.1  # Initial move scale
    angle_scale = 30.0  # Initial angle scale
    
    for it in range(iterations):
        # Scale moves with temperature
        sc = T / T0
        
        # Choose random tree to move
        i = np.random.randint(n)
        
        # Save old position
        old_pos = list(current_positions[i])
        
        # Make random move
        move_type = np.random.randint(3)
        if move_type == 0:  # Translate
            current_positions[i][0] += (np.random.random() - 0.5) * 2 * move_scale * sc
            current_positions[i][1] += (np.random.random() - 0.5) * 2 * move_scale * sc
        elif move_type == 1:  # Rotate
            current_positions[i][2] += (np.random.random() - 0.5) * 2 * angle_scale * sc
            current_positions[i][2] = current_positions[i][2] % 360
        else:  # Both
            current_positions[i][0] += (np.random.random() - 0.5) * move_scale * sc
            current_positions[i][1] += (np.random.random() - 0.5) * move_scale * sc
            current_positions[i][2] += (np.random.random() - 0.5) * angle_scale * sc
            current_positions[i][2] = current_positions[i][2] % 360
        
        # Check if move violates gap constraint
        if has_gap_violation(current_positions):
            current_positions[i] = old_pos
            T *= alpha
            continue
        
        # Calculate new score
        new_score = get_score(current_positions)
        delta = new_score - current_score
        
        # Accept or reject
        if delta < 0 or np.random.random() < np.exp(-delta / T):
            current_score = new_score
            if new_score < best_score:
                best_score = new_score
                best_positions = [list(p) for p in current_positions]
        else:
            current_positions[i] = old_pos
        
        T *= alpha
    
    return best_positions, best_score

print("Gap-constrained SA defined")

Gap-constrained SA defined


In [3]:
# Test on small N values
print("Testing gap-constrained SA on small N values...")

for test_n in [2, 3, 4, 5]:
    print(f"\nN={test_n}:")
    result = gap_constrained_sa(test_n, iterations=3000)
    if result:
        positions, score = result
        trees = [ChristmasTree(p[0], p[1], p[2]) for p in positions]
        min_dist = get_min_distance(trees)
        contribution = (score ** 2) / test_n
        print(f"  Score: {score:.6f}, min_dist: {min_dist:.6f}, contribution: {contribution:.6f}")
        print(f"  Valid: {is_valid_configuration(trees, min_gap=1e-9)}")

Testing gap-constrained SA on small N values...

N=2:


  Score: 1.609576, min_dist: 0.068753, contribution: 1.295367
  Valid: True

N=3:


  Score: 1.722566, min_dist: 0.042155, contribution: 0.989078
  Valid: True

N=4:


  Score: 2.316414, min_dist: 0.130968, contribution: 1.341443
  Valid: True

N=5:


  Score: 2.911148, min_dist: 0.255571, contribution: 1.694956
  Valid: True


In [ ]:
# Load valid ensemble for comparison
df_valid = pd.read_csv('/home/code/experiments/002_valid_submission/submission.csv')
df_touching = pd.read_csv('/home/code/experiments/002_valid_ensemble/submission.csv')

print("Comparing with existing solutions for small N:")
for n in range(1, 11):
    trees_valid = load_trees_for_n(df_valid, n)
    trees_touching = load_trees_for_n(df_touching, n)
    
    valid_side = get_bounding_box_side(trees_valid)
    touching_side = get_bounding_box_side(trees_touching)
    
    valid_contrib = (valid_side ** 2) / n
    touching_contrib = (touching_side ** 2) / n
    
    print(f"N={n:2d}: valid={valid_contrib:.4f}, touching={touching_contrib:.4f}, gap={valid_contrib - touching_contrib:.4f}")

In [ ]:
# Run gap-constrained SA on N=1-20 and compare with valid ensemble
print("Running gap-constrained SA on N=1-20...")

results = {}
for n in range(1, 21):
    # Get valid ensemble as starting point
    trees_valid = load_trees_for_n(df_valid, n)
    initial_positions = [(t.center_x, t.center_y, t.angle) for t in trees_valid]
    
    # Run SA
    result = gap_constrained_sa(n, initial_positions=initial_positions, iterations=5000)
    
    if result:
        positions, score = result
        trees = [ChristmasTree(p[0], p[1], p[2]) for p in positions]
        min_dist = get_min_distance(trees)
        contribution = (score ** 2) / n
        
        # Compare with valid ensemble
        valid_side = get_bounding_box_side(trees_valid)
        valid_contrib = (valid_side ** 2) / n
        
        improvement = valid_contrib - contribution
        
        results[n] = {
            'positions': positions,
            'side': score,
            'contribution': contribution,
            'valid_contribution': valid_contrib,
            'improvement': improvement,
            'min_dist': min_dist
        }
        
        if improvement > 0.0001:
            print(f"N={n:2d}: SA={contribution:.4f}, valid={valid_contrib:.4f}, improvement={improvement:.4f}")
    else:
        # Use valid ensemble
        valid_side = get_bounding_box_side(trees_valid)
        valid_contrib = (valid_side ** 2) / n
        results[n] = {
            'positions': initial_positions,
            'side': valid_side,
            'contribution': valid_contrib,
            'valid_contribution': valid_contrib,
            'improvement': 0,
            'min_dist': get_min_distance(trees_valid)
        }
        print(f"N={n:2d}: SA failed, using valid ensemble")

print(f"\nTotal improvement for N=1-20: {sum(r['improvement'] for r in results.values()):.6f}")

In [ ]:
# Build full submission: Use SA results for N=1-20, valid ensemble for N=21-200
print("Building full submission...")

all_results = {}

# N=1-20: Use SA results
for n in range(1, 21):
    all_results[n] = results[n]

# N=21-200: Use valid ensemble
for n in range(21, 201):
    trees_valid = load_trees_for_n(df_valid, n)
    valid_side = get_bounding_box_side(trees_valid)
    valid_contrib = (valid_side ** 2) / n
    all_results[n] = {
        'positions': [(t.center_x, t.center_y, t.angle) for t in trees_valid],
        'side': valid_side,
        'contribution': valid_contrib
    }

# Calculate total score
total_score = sum(r['contribution'] for r in all_results.values())
print(f"\nTotal score: {total_score:.6f}")
print(f"Valid ensemble score: 71.812779")
print(f"Improvement: {71.812779 - total_score:.6f}")

In [ ]:
# Verify all configurations are valid
print("Verifying validity...")
invalid_count = 0
for n in range(1, 201):
    trees = [ChristmasTree(p[0], p[1], p[2]) for p in all_results[n]['positions']]
    if not is_valid_configuration(trees, min_gap=1e-9):
        invalid_count += 1
        print(f"N={n}: INVALID (min_dist={get_min_distance(trees):.2e})")

if invalid_count == 0:
    print("All configurations are valid!")
else:
    print(f"\n{invalid_count} invalid configurations")

In [ ]:
# Build and save submission
all_rows = []
for n in range(1, 201):
    for idx, (x, y, angle) in enumerate(all_results[n]['positions']):
        all_rows.append({
            'id': f"{n:03d}_{idx}",
            'x': f"s{x}",
            'y': f"s{y}",
            'deg': f"s{angle}"
        })

submission_df = pd.DataFrame(all_rows)
print(f"Submission has {len(submission_df)} rows")

# Save
submission_df.to_csv('/home/code/experiments/005_gap_constrained_cpp/submission.csv', index=False)
print("Saved to experiments/005_gap_constrained_cpp/submission.csv")

import shutil
shutil.copy('/home/code/experiments/005_gap_constrained_cpp/submission.csv', '/home/submission/submission.csv')
print("Copied to /home/submission/submission.csv")

metrics = {'cv_score': total_score}
with open('/home/code/experiments/005_gap_constrained_cpp/metrics.json', 'w') as f:
    json.dump(metrics, f)
print(f"Metrics saved: {metrics}")